## 自动分割 3D

ROI区域自动分割，数据的一般形式为：

* images文件夹，存放研究对象所有的CT、MRI等数据。
* masks文件夹, 存放手工（Manuelly）勾画的ROI区域。与images文件夹的文件一一对应。

**注意**：`my_dir`下必须是包括images和masks文件夹。

In [3]:
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from pixelmed_calc.segmentation3D.modelzoo.run_3dsegmentation import main as seg_main

# 如果自己有coco格式的数据，可以直接使用自己的目录。
my_dir = r'C:\Users\yxzdxzj\Desktop\function\pm_data\MR'

# 设置参数
class params:
    train = my_dir                    # 训练数据的目录
    valid = None                      # 测试数据的目录，如果为None，则使用val_size自动划分，否则val_size参数不生效。
    val_size = 0.1                    # 训练集划分出来作为测试集的比例。
    roi_size = [48, 48, 16]           # ROI的预估大小。
    num_classes = 2                   # 识别的ROI的类别数。
    j = 0                             # 并行载入数据的并发量，默认0，如果CPU足够强劲可以选择 > 1.
    model_name = 'Unet'               # 模型名称，目前支持Unet、Segres、Unetr
    init_lr = 0.001                   # 初始化的learning rate。
    batch_size = 4                    # 每次训练的batch_size大小
    iters_verbose = 10                # 打印log的频率
    epochs = 3                        # 训练的总次数。
    optimizer = 'Adamax'              # 优化器的选择。SGD、Adam、Adamax
    val_interval = 4                  # 多少次训练进行一次evaluate
    cached_ratio=0                    # 缓存数据的比例
    gpu = '0'                          # 如果有GPU，确定使用的GPU ID，一般大家填0
    save_dir = '.'                    # 模型保存的位置。
    attr = {}

    def __setattr__(self, key, value):
        self.attr[key] = value


# 训练模型
seg_main(params)

[2023-09-14 12:11:32 - run_3dsegmentation.py:  79]	INFO	使用自定义Transform
[2023-09-14 12:11:32 - run_3dsegmentation.py:  95]	INFO	一共获取214个训练样本，23个测试样本。
[2023-09-14 12:11:32 - utils.py: 150]	INFO	使用数据缓存，缓存比例：0


epoch: 1, step:10, train_loss: 1.0958, lr: 0.000990, step time: 0.4726
epoch: 1, step:20, train_loss: 1.0445, lr: 0.000962, step time: 0.3926
epoch: 1, step:30, train_loss: 0.9236, lr: 0.000916, step time: 0.4500
epoch: 1, step:40, train_loss: 0.8692, lr: 0.000854, step time: 0.4525
epoch: 1, step:50, train_loss: 0.8319, lr: 0.000778, step time: 0.3593
epoch 1 average loss: 0.9839
saved new best metric model
current epoch: 1 current mean dice: 0.0102
best mean dice: 0.0102 at epoch: 1
epoch: 2, step:10, train_loss: 0.7331, lr: 0.000645, step time: 0.4062
epoch: 2, step:20, train_loss: 0.8108, lr: 0.000549, step time: 0.3841
epoch: 2, step:30, train_loss: 0.7221, lr: 0.000451, step time: 0.3905
epoch: 2, step:40, train_loss: 0.7677, lr: 0.000355, step time: 0.4061
epoch: 2, step:50, train_loss: 0.6852, lr: 0.000264, step time: 0.4549
epoch 2 average loss: 0.7416
epoch: 3, step:10, train_loss: 0.6848, lr: 0.000146, step time: 0.3896
epoch: 3, step:20, train_loss: 0.7448, lr: 0.000084, st

### 预测模型

需要确定自己的预测数据，修改my_test_dir参数。

In [4]:
import glob
import os

from pixelmed_calc.segmentation3D.modelzoo.eval_3dsegmentation import init as init3d
from pixelmed_calc.segmentation3D.modelzoo.eval_3dsegmentation import inference as inference3d

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
my_test_dir = r'C:\Users\yxzdxzj\Desktop\function\pm_data\MR\deeplearning\val\label1'
data = [i for i in glob.glob(os.path.join(my_test_dir, '*.nii*'))]
m, t, d = init3d('Unet', model_path='20230914/Unet/Unet.pth', num_classes=2)
inference3d(data, m, t, d, roi_size=(48, 48, 48), save_dir='.')

[2023-09-14 12:17:19 - eval_3dsegmentation.py:  90]	INFO	使用Unet模型，成功加载20230914/Unet/Unet.pth参数
[2023-09-14 12:17:19 - eval_3dsegmentation.py:  61]	INFO	正在预测{'image': 'C:\\Users\\yxzdxzj\\Desktop\\function\\pm_data\\MR\\deeplearning\\val\\label1\\11.nii.gz'}
[2023-09-14 12:17:24 - eval_3dsegmentation.py:  61]	INFO	正在预测{'image': 'C:\\Users\\yxzdxzj\\Desktop\\function\\pm_data\\MR\\deeplearning\\val\\label1\\12.nii.gz'}
[2023-09-14 12:17:27 - eval_3dsegmentation.py:  61]	INFO	正在预测{'image': 'C:\\Users\\yxzdxzj\\Desktop\\function\\pm_data\\MR\\deeplearning\\val\\label1\\13.nii.gz'}
[2023-09-14 12:17:31 - eval_3dsegmentation.py:  61]	INFO	正在预测{'image': 'C:\\Users\\yxzdxzj\\Desktop\\function\\pm_data\\MR\\deeplearning\\val\\label1\\14.nii.gz'}
[2023-09-14 12:17:36 - eval_3dsegmentation.py:  61]	INFO	正在预测{'image': 'C:\\Users\\yxzdxzj\\Desktop\\function\\pm_data\\MR\\deeplearning\\val\\label1\\15.nii.gz'}
